# **A/B test - Automatidata**

<hr style="height: 3px; border:none; color:#000; background-color:#000;" />


This report presents an analysis of taxi trip data to evaluate key relationships and differences across various trip and payment characteristics. Using hypothesis testing as the primary method, we aim to uncover statistically significant patterns and insights

In [34]:
# Libraries
import numpy as np
import pandas as pd

from scipy import stats

## **1. Load Data and General Exploratory Analysis**

In [35]:
df_taxi = pd.read_csv("Datasets/2017_Yellow_Taxi_Trip_Data.csv")
df_taxi.head(10)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,24870114,2,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,6,3.34,1,N,100,231,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56
1,35634249,1,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,1,1.80,1,N,186,43,1,16.0,0.0,0.5,4.00,0.0,0.3,20.80
2,106203690,1,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,1,1.00,1,N,262,236,1,6.5,0.0,0.5,1.45,0.0,0.3,8.75
3,38942136,2,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,1,3.70,1,N,188,97,1,20.5,0.0,0.5,6.39,0.0,0.3,27.69
4,30841670,2,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,1,4.37,1,N,4,112,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80
5,23345809,2,03/25/2017 8:34:11 PM,03/25/2017 8:42:11 PM,6,2.30,1,N,161,236,1,9.0,0.5,0.5,2.06,0.0,0.3,12.36
6,37660487,2,05/03/2017 7:04:09 PM,05/03/2017 8:03:47 PM,1,12.83,1,N,79,241,1,47.5,1.0,0.5,9.86,0.0,0.3,59.16
7,69059411,2,08/15/2017 5:41:06 PM,08/15/2017 6:03:05 PM,1,2.98,1,N,237,114,1,16.0,1.0,0.5,1.78,0.0,0.3,19.58
8,8433159,2,02/04/2017 4:17:07 PM,02/04/2017 4:29:14 PM,1,1.20,1,N,234,249,2,9.0,0.0,0.5,0.00,0.0,0.3,9.80
9,95294817,1,11/10/2017 3:20:29 PM,11/10/2017 3:40:55 PM,1,1.60,1,N,239,237,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55


In [36]:
df_taxi = df_taxi.set_index('Unnamed: 0')
df_taxi.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
Unnamed: 0,,,,,,,,,,,,,,,,,
24870114,2,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,6,3.34,1,N,100,231,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56
35634249,1,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,1,1.80,1,N,186,43,1,16.0,0.0,0.5,4.00,0.0,0.3,20.80
106203690,1,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,1,1.00,1,N,262,236,1,6.5,0.0,0.5,1.45,0.0,0.3,8.75
38942136,2,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,1,3.70,1,N,188,97,1,20.5,0.0,0.5,6.39,0.0,0.3,27.69
30841670,2,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,1,4.37,1,N,4,112,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80
23345809,2,03/25/2017 8:34:11 PM,03/25/2017 8:42:11 PM,6,2.30,1,N,161,236,1,9.0,0.5,0.5,2.06,0.0,0.3,12.36
37660487,2,05/03/2017 7:04:09 PM,05/03/2017 8:03:47 PM,1,12.83,1,N,79,241,1,47.5,1.0,0.5,9.86,0.0,0.3,59.16
69059411,2,08/15/2017 5:41:06 PM,08/15/2017 6:03:05 PM,1,2.98,1,N,237,114,1,16.0,1.0,0.5,1.78,0.0,0.3,19.58
8433159,2,02/04/2017 4:17:07 PM,02/04/2017 4:29:14 PM,1,1.20,1,N,234,249,2,9.0,0.0,0.5,0.00,0.0,0.3,9.80


In [37]:
df_taxi.info()


<class 'pandas.core.frame.DataFrame'>
Index: 22699 entries, 24870114 to 17208911
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               22699 non-null  int64  
 1   tpep_pickup_datetime   22699 non-null  object 
 2   tpep_dropoff_datetime  22699 non-null  object 
 3   passenger_count        22699 non-null  int64  
 4   trip_distance          22699 non-null  float64
 5   RatecodeID             22699 non-null  int64  
 6   store_and_fwd_flag     22699 non-null  object 
 7   PULocationID           22699 non-null  int64  
 8   DOLocationID           22699 non-null  int64  
 9   payment_type           22699 non-null  int64  
 10  fare_amount            22699 non-null  float64
 11  extra                  22699 non-null  float64
 12  mta_tax                22699 non-null  float64
 13  tip_amount             22699 non-null  float64
 14  tolls_amount           22699 non-null  float64
 1

In [38]:
# Generate a table of descriptive statistics about the data
df_taxi.describe(include='all')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,22699.000000,22699,22699,22699.000000,22699.000000,22699.000000,22699,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000
unique,NaN,22687,22688,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,07/03/2017 3:45:19 PM,10/18/2017 8:07:45 PM,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,2,NaN,NaN,NaN,22600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.556236,NaN,NaN,1.642319,2.913313,1.043394,NaN,162.412353,161.527997,1.336887,13.026629,0.333275,0.497445,1.835781,0.312542,0.299551,16.310502
std,0.496838,NaN,NaN,1.285231,3.653171,0.708391,NaN,66.633373,70.139691,0.496211,13.243791,0.463097,0.039465,2.800626,1.399212,0.015673,16.097295
min,1.000000,NaN,NaN,0.000000,0.000000,1.000000,NaN,1.000000,1.000000,1.000000,-120.000000,-1.000000,-0.500000,0.000000,0.000000,-0.300000,-120.300000
25%,1.000000,NaN,NaN,1.000000,0.990000,1.000000,NaN,114.000000,112.000000,1.000000,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,8.750000
50%,2.000000,NaN,NaN,1.000000,1.610000,1.000000,NaN,162.000000,162.000000,1.000000,9.500000,0.000000,0.500000,1.350000,0.000000,0.300000,11.800000
75%,2.000000,NaN,NaN,2.000000,3.060000,1.000000,NaN,233.000000,233.000000,2.000000,14.500000,0.500000,0.500000,2.450000,0.000000,0.300000,17.800000


In [39]:
# Check for missing values
df_taxi.isna().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

## **2. Variable Selection for A/B Testing:**
Based on the exploratory analysis, the following variables were chosen for A/B testing due to their relevance and potential to impact the analysis:
- **Payment_type:** To evaluate how the payment method (e.g., credit card, cash) influences other variables like Total_amount and Tip_amount.
- **Total_amount:** Represents the total fare charged to passengers, a key metric for assessing the financial outcomes of trips.
- **VendorID:** Indicates the taxi provider, useful for comparing performance or service differences between vendors.
- **Trip_distance:** Captures the distance of the trip, which can be analyzed in relation to payment methods or total fares.
- **Tip_amount:** Provides insight into tipping behavior, which could vary by payment method or trip characteristics.

### **Categorical Variables**

In [40]:
# Display unique values for categorical variables (selected)
print("Unique values for the 'payment_type'")
print(f'-> {df_taxi["payment_type"].unique()}')
print()
print("Unique values for the 'VendorID'")
print(f'-> {df_taxi["VendorID"].unique()}')

Unique values for the 'payment_type'
-> [1 2 3 4]

Unique values for the 'VendorID'
-> [2 1]


### **Continuous Variables**

Regarding the variable **fare_amount**, it has been identified that it contains anomalous values (negative values) where they should be greater than 0 (positive values)

In [41]:
df_taxi["fare_amount"].describe()

count    22699.000000
mean        13.026629
std         13.243791
min       -120.000000
25%          6.500000
50%          9.500000
75%         14.500000
max        999.990000
Name: fare_amount, dtype: float64

In [42]:
df_taxi[df_taxi["fare_amount"]<0].count()

VendorID                 14
tpep_pickup_datetime     14
tpep_dropoff_datetime    14
passenger_count          14
trip_distance            14
RatecodeID               14
store_and_fwd_flag       14
PULocationID             14
DOLocationID             14
payment_type             14
fare_amount              14
extra                    14
mta_tax                  14
tip_amount               14
tolls_amount             14
improvement_surcharge    14
total_amount             14
dtype: int64

Out of a total of 22,699 instances, only 14 have a "fare_amount" less than 0. These anomalous values will be filtered out, and a new DataFrame will be created without them to ensure the reliability of the analysis

In [43]:
df2_taxi = df_taxi[df_taxi["fare_amount"]>=0]
df2_taxi["fare_amount"].describe()

count    22685.000000
mean        13.041876
std         13.212569
min          0.000000
25%          6.500000
50%          9.500000
75%         14.500000
max        999.990000
Name: fare_amount, dtype: float64

## **3.  Hypothesis Testing:**
This section outlines the hypotheses formulated to analyze differences between groups in the dataset. Each hypothesis includes a null hypothesis (H₀) and an alternative hypothesis (H₁) to evaluate significant relationships among key variables:

**Case 1:Analysis of Total Amount Differences by Payment Method (Credit Card and Cash)**
- $H_0$: There is no significant difference in the total amount (Total_amount) between trips paid with credit card and those paid with cash
- $H_A$: There is a significant difference in the total amount (Total_amount) between trips paid with credit card and those paid with cash.

**Case 2: Analysis of Tip Amount Differences by Payment Method (Credit Card and Cash)**
- $H_0$: There is no difference in the average tip amount (Tip_amount) between trips paid with credit card and those paid with cash.
- $H_A$: There is a difference in the average tip amount (Tip_amount) between trips paid with credit card and those paid with cash.

**Case 3: Analysis of Trip Distance Differences by Vendor**
- $H_0$: There is no significant difference in the average trip distance (Trip_distance) between trips by Creative Mobile Technologies (VendorID = 1) and VeriFone Inc. (VendorID = 2).
- $H_A$: There is a significant difference in the average trip distance (Trip_distance) between trips by Creative Mobile Technologies (VendorID = 1) and VeriFone Inc. (VendorID = 2).

## **4.  Perform A/B Testing:**
<p style="text-align: justify;">
This section presents the application of hypothesis testing to evaluate differences in key metrics across different groups. Three specific cases are analyzed: (1) differences in total fare amounts between payment methods (Credit Card and Cash), (2) differences in tip amounts by payment method, and (3) differences in trip distances between vendors. For each case, Levene's Test is used to assess the homogeneity of variances, followed by a Two-Sample T-Test (with Welch’s correction if needed) to determine whether the observed differences in means are statistically significant </p>

**Considerations:**
- A significance level of 5% will be used for the analysis.
- To perform hypothesis testing, the assumptions of normality and homogeneity of variances should be evaluated.
- For this analysis, it is assumed that the normality condition is met due to the large sample size, relying on the Central Limit Theorem.

### **4.1. Case 1:Analysis of Total Amount Differences by Payment Method (Credit Card and Cash)**

Preliminary Analysis: Average 'fare_amount' by Payment Method

In [44]:
df2_taxi.groupby('payment_type')[['fare_amount']].mean(numeric_only=True)

,fare_amount
payment_type,
1,13.429748
2,12.213546
3,13.127368
4,15.320513


Data Preparation: Splitting 'fare_amounts' by Payment Method

In [45]:
sig_level = 0.05
credit_card = df2_taxi[df2_taxi['payment_type'] == 1]['fare_amount']
cash = df2_taxi[df2_taxi['payment_type'] == 2]['fare_amount']

Before performing the Two-Sample t-test, we evaluated the assumption of homogeneity of variances (equal variance).

In [46]:
stat, p = stats.levene(credit_card, cash)
print(f"Levene’s Test: stat = {stat}, p-value = {p}")

# Check if the variances are equal (p-value >= 0.05)
if p >= 0.05:
    print("-> Homogeneity of variances (equal variances) is met (p >= 0.05).")
else:
    print("-> Homogeneity of variances (equal variances) is not met (p < 0.05).")

Levene’s Test: stat = 9.68147691951664, p-value = 0.0018636752552398772
-> Homogeneity of variances (equal variances) is not met (p < 0.05).


In [47]:
# Perform Two Sample t-test
tstatistic, pvalue= stats.ttest_ind(a=credit_card, b=cash, equal_var=False)
print("Results:")
print(f'T-statistic: {tstatistic}')
print(f'P-value: {pvalue}')

Results:
T-statistic: 6.866800855655372
P-value: 6.797387473030518e-12


**Conclusion:** The p-value is far below the significance level of 0.05, so we reject the null hypothesis ($H_0$). This indicates that there is a statistically significant difference in the total amount between trips paid with credit card and those paid with cash.

### **4.2. Case 2: Analysis of Tip Amount Differences by Payment Method (Credit Card and Cash)**

Preliminary Analysis: Average 'tip_amount' by Payment Method

In [48]:
df2_taxi.groupby("payment_type")[['tip_amount']].mean(numeric_only=True)

,tip_amount
payment_type,
1,2.7298
2,0.0000
3,0.0000
4,0.0000


**Conclusion:** The analysis of tip amounts by payment type revealed that payment type 1 had an average tip amount of 2.7298, while payment type 2 recorded no tips (average = 0). Given the lack of variability in tip amounts for payment type 2, a hypothesis test such as the t-test was deemed unnecessary, as the results are already conclusive based on descriptive statistics alone

### **4.3. Case 3: Analysis of Trip Distance Differences by Vendor**

In [49]:
df2_taxi.groupby("VendorID")[["trip_distance"]].mean(numeric_only=True)

,trip_distance
VendorID,
1,2.881485
2,2.941681


Data Preparation: Splitting 'trip_distance' by Vendor

In [50]:
sig_level = 0.05
vendor1 = df2_taxi[df2_taxi["VendorID"] == 1]["trip_distance"]
vendor2 = df2_taxi[df2_taxi["VendorID"] == 2]["trip_distance"]

Before performing the Two-Sample t-test, we evaluated the assumption of homogeneity of variances (equal variance).

In [51]:
stat, p = stats.levene(vendor1, vendor2)
print(f"Levene’s Test: stat = {stat}, p-value = {p}")

# Check if the variances are equal (p-value >= 0.05)
if p >= 0.05:
    print("-> Homogeneity of variances (equal variances) is met (p >= 0.05).")
else:
    print("-> Homogeneity of variances (equal variances) is not met (p < 0.05).")

Levene’s Test: stat = 0.2610759197504553, p-value = 0.6093869182179863
-> Homogeneity of variances (equal variances) is met (p >= 0.05).


In [52]:
# Perform Two Sample t-test
tstatistic, pvalue= stats.ttest_ind(a=vendor1, b=vendor2, equal_var=True)
print("Results:")
print(f'T-statistic: {tstatistic}')
print(f'P-value: {pvalue}')

Results:
T-statistic: -1.2329354803682782
P-value: 0.21761261424712913


**Conclusions:** Since the p-value exceeds the significance level of 0.05, we fail to reject the null hypothesis. This suggests that there is no statistically significant difference in the average trip distances between the two vendors.

## **5.  CONCLUSIONS AND RECOMMENDATIONS:**

- **Impact of Payment Method on Total Amount:** A statistically significant difference was found in the total fare amounts between trips paid with credit card and those paid with cash. This indicates that payment method influences the overall fare charged.
- **Influence of Payment Method on Tip Amount:** Trips paid with credit card recorded significantly higher tip amounts compared to cash payments, where tips were non-existent. This highlights a strong association between payment method and customer tipping behavior.
- **Comparison of Trip Distances Between Vendors:** No statistically significant difference was observed in the average trip distances between the two vendors, suggesting that both providers offer comparable trip characteristics in terms of distance.
- It is recommended to test whether the average fare amount (Total_amount) differs among 'RateCodeID' (e.g., standard, JFK, Newark) to evaluate pricing variations by trip type.
- It is recommended to test whether trips originating from high-demand zones (e.g., airports) result in higher average fares compared to other zones.